# Data Processing Example

In [ ]:
from fibermagic.IO.NeurophotometricsIO import read_project_rawdata, read_project_logs
from pathlib import Path
import plotly.express as px
from fibermagic.core.demodulate import zdFF_airPLS, add_zdFF
from fibermagic.core.perievents import perievents
from fibermagic.utils.download_dataset import download

In [ ]:
download('arena-data')

The data was taken in two batches from different mice. Batch 1 consists of one recording per mouse (n=2). Batch 2 includes two recordings per mouse (n=7). Every mouse was recorded individually (no multiple mice per recording).

The experimental environment was a 1m x 1m sized arena. The mouse was able to move freely. Food pellets, a running wheel, and a novel object was placed in different locations of the arena. Sessions were 15 minutes long.

For behavioral evaluation, a video was recorded using a IR-camera. The video was used to determing the region in which the mouse was.

In [ ]:
PROJECT_PATH = Path(r'data')
df_idx = read_project_rawdata(PROJECT_PATH,
                     ['Batch', 'Day', 'FP', 'Mouse_'], 'arena.csv', ignore_dirs=['meta', 'processed', 'processed.zip'])

### A first spike into to investigate the raw data.

By changing values to different mice and days, you can see that mice B9671 and B9889 had bad signals.

In [ ]:
df = df_idx.reset_index()
df.Mouse.unique()  # list recorded mice

In [ ]:
px.line(df[(df.Mouse=='B9890') & (df.Day=='Day2') & (df.Channel == 560)].Signal).show()

In [ ]:
px.line(df[(df.Mouse=='B9890') & (df.Day=='Day2') & (df.Channel == 560)].Reference)

In [ ]:
df_idx = add_zdFF(df_idx, smooth_win=10, remove=200, lambd=5e9).set_index('FrameCounter', append=True)
df_idx

### Look at eating behavior in the full session

In [ ]:
df = df_idx.reset_index()
subset = df[(df.Mouse=='B9890') & (df.Day=='Day2') ]
food_intake_9890 = [2277, 2516, 2692, 3234, 3548, 3712, 3936, 4315, 4483, 4671, 6323, 6418, 6589, 6659, 6853, 7590, 8890]
fig = px.line(x=(subset.FrameCounter - 297)/2.5, y=subset['zdFF (airPLS)'], color=subset.Channel)
for food in food_intake_9890:
    fig = fig.add_vline(x=food)
fig.show()

In [ ]:
food_intake_9670 = [195, 289, 324, 362, 739, 781, 1262, 1330, 1454, 2130, 2190, 2310, 2366, 3075, 4022, 4080, 4180, 4240, 4350, 4480, 5140, 7410, 8094, 8150]
subset = df[(df.Mouse=='B9670') & (df.Day=='Day2') & (df.Channel==560) ]
fig = px.line(x=(subset.FrameCounter - 307)/2.5, y=subset['zdFF (airPLS)'])
for food in food_intake_9890:
    fig = fig.add_vline(x=food)
fig.show()

### Synchronize photometry recording with behavioral events from video recording

In [ ]:
import pandas as pd

def arena_sync(logs, path):
    ttl = pd.read_csv(path/'input1.csv')
    frame_times = pd.read_csv(path/'time.csv')
    first_ttl = ttl.loc[0, 'Item2']
    
    first_frame = ((first_ttl - frame_times.Item2).abs().idxmin())
    framerate = 30
    logs['Timestamp'] = logs.Frame / 30
    logs['FrameCounter'] = logs.Timestamp * 25 // 1
    logs['FrameCounter'] += first_frame
    logs['FrameCounter'] = logs.FrameCounter.astype(int)
    logs = logs.rename(columns={'ROI': 'Event'})
    
    # ignore events that are too close to start or end of recording
    logs = logs[(logs.Timestamp > 20) & (logs.Timestamp < max(logs.Timestamp) - 20)]
    return logs.set_index('FrameCounter')


In [ ]:
import os
logs = read_project_logs(PROJECT_PATH,
                          ['Batch', 'Day', 'FP', 'Mouse_'], sync_fun=arena_sync, ignore_dirs=['meta', 'processed', 'processed.zip'])
logs

### Select appropriate events and calculate perievents

In [ ]:
# use only entries if mouse was more than x seconds in region
min_sec_in_region = 10

idxin = None
onset = 0
to_drop = list()
for index, row in logs.iterrows():
    if '_in' in row.Event:
        idxin = index
        onset = row.Timestamp
    elif '_out' in row.Event:
        if row.Timestamp - onset < min_sec_in_region:
            to_drop.append(idxin)
            to_drop.append(index)
logs = logs.drop(index=to_drop)

In [ ]:
peri = perievents(df_idx, logs, 15, 25)
peri

# Further Analysis

In [ ]:
# save datastream, logs and perievents

output_path = PROJECT_PATH / 'processed'

# uncomment if you like to save the output files
# df.to_csv(output_path / 'datastream.csv')
# logs.to_csv(output_path / 'logs_time_criteria_synced.csv')
# peri.to_csv(output_path / 'perievents_time_criteria_synced.csv')

Now, you can use data analysis tools like Tableau or PowerBI or any data analysis technology like R, Matlab, Python (e.g. Seaborn) to visualize your data